In [4]:
import pandas as pd

df_matched_all = pd.read_csv('df_after_match_15_unified.csv')
# df_new_anomalies_all = pd.read_csv('df_new_anomalies_all')
df = df_matched_all.copy()
# df['T_C'] = np.random.uniform(28, 31, size=1046)
df['temperature_C'] = 30.8
df['p_H2S_bar'] = 0

import numpy as np

n = len(df) 


# mask = np.random.rand(n) < 0.2  

# p_vals = np.empty(n)


# p_vals[mask] = np.random.uniform(70, 75, size=mask.sum())
# p_vals[~mask] = np.random.uniform(90, 95, size=(~mask).sum())


df['pressure_bar'] = 92


df['water_flag'] = 1

In [5]:
df

,latitude,longitude,heighting,angle_deg,depth_cgr,feature_identification,anomaly_dim_class,depth,length,width,erf_b31g,temperature_C,p_H2S_bar,pressure_bar,water_flag
0,16.736072,82.293731,4.931,120.5,0.000,Corrosion,PINH,2.54,15.0,18.0,0.91,30.8,0,92,1
1,16.736026,82.293298,5.762,254.0,0.000,Pipe mill anomaly,AXSL,2.54,40.0,23.0,0.91,30.8,0,92,1
2,16.735664,82.289963,8.687,169.0,0.635,Pipe mill anomaly,AXSL,2.54,60.0,18.0,0.91,30.8,0,92,1
3,16.736874,82.286373,10.197,71.5,0.000,Pipe mill anomaly,PINH,2.54,22.0,23.0,0.91,30.8,0,92,1
4,16.736874,82.286373,10.197,71.5,0.000,Pipe mill anomaly,PINH,2.54,22.0,23.0,0.91,30.8,0,92,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,16.753963,81.017837,34.964,233.5,0.430,Pipe mill anomaly,AXGR,1.72,45.0,21.0,0.91,30.8,0,92,1
1102,16.752867,81.015595,40.502,136.5,0.430,Pipe mill anomaly,PITT,1.72,31.0,20.0,0.91,30.8,0,92,1
1103,16.752521,81.014897,41.232,310.0,0.430,Pipe mill anomaly,PITT,1.72,57.0,35.0,0.91,30.8,0,92,1
1104,16.752417,81.014690,41.183,58.0,0.430,Pipe mill anomaly,PITT,1.72,29.0,18.0,0.91,30.8,0,92,1


### Using physics + AI based hybrid model

In [6]:
import numpy as np
import pandas as pd


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GroupKFold
from xgboost import XGBRegressor




def dew_point_C_from_pH2O_kPa(p_h2o_kpa):
    p = np.clip(np.asarray(p_h2o_kpa, dtype=float), 1e-6, None)
    ln_arg = np.log(p / 0.61078)
    return (237.3 * ln_arg) / (17.27 - ln_arg)

def compute_water_flag(P_bar, y_H2O, T_wall_C, margin_C=2.0):
    p_h2o_kpa = np.clip(y_H2O, 0.0, 1.0) * np.clip(P_bar, 0.0, None) * 100.0  # bar→kPa
    T_dp = dew_point_C_from_pH2O_kPa(p_h2o_kpa)
    return (np.asarray(T_wall_C) <= (T_dp - margin_C)).astype(int), T_dp

def f_co2_mpa(P_bar, y_CO2, z):
    phi = np.clip(np.asarray(z), 0.3, 1.2)
    P_mpa = np.clip(np.asarray(P_bar), 0.0, None) / 10.0  # bar→MPa
    return np.clip(phi * np.asarray(y_CO2) * P_mpa, 1e-5, None)

def T_scale_K(f_mpa):
    f = np.clip(np.asarray(f_mpa), 1e-5, None)
    return 2400.0 / (6.7 + 0.61*np.log10(f))

def C_R_star_mm_y(T_K, f_mpa, gamma=0.03, A_log=1710.0, C0_log=5.8, fexp_log=0.67):
    T = np.clip(np.asarray(T_K), 1e-6, None)
    f = np.clip(np.asarray(f_mpa), 1e-5, None)
    base_log10 = C0_log - A_log/T + fexp_log*np.log10(f)
    CR = 10.0**base_log10
    decay = np.exp(-gamma * np.maximum(T - T_scale_K(f), 0.0))
    return CR * decay

def C_MT_simple_mm_y(U_ms, diameter_m, f_mpa, c1=0.2, b=0.7, c=0.0, betaMT=0.4):
    U = np.clip(np.asarray(U_ms), 1e-6, None)
    d = np.clip(np.asarray(diameter_m), 1e-6, None)
    f = np.clip(np.asarray(f_mpa), 1e-5, None)
    return c1 * (U**b) * (d**c) * (f**betaMT)

def C_phys_mm_y(T_K, f_mpa, water_flag, U_ms, diameter_m,
                gamma=0.03, c1=0.2, b=0.7, c=0.0, betaMT=0.4,
                CpH=1.0, Cscale=1.0, Ctau=1.0):
    CR  = C_R_star_mm_y(T_K, f_mpa, gamma=gamma)
    CMT = C_MT_simple_mm_y(U_ms, diameter_m, f_mpa, c1=c1, b=b, c=c, betaMT=betaMT)
    CR  = np.clip(CR,  1e-12, None)
    CMT = np.clip(CMT, 1e-12, None)
    Ccomb = 1.0 / (1.0/CR + 1.0/CMT)
    return Ccomb * np.asarray(water_flag) * CpH * Cscale * Ctau


# 1 )Feature engineering

def add_engineered_features(df, params_phys=None, defaults=None):
    """
    Adds physics & encodings needed by the ML model.
    params_phys: dict with gamma, c1, b, c, betaMT (optional, to compute C_phys baseline as a feature)
    defaults: dict with fallback values for z, y_CO2, y_H2O, U_ms, diameter_m
    """
    out = df.copy()

    # Defaults (you can pass pipeline-level constants here)
    defaults = defaults or {}
    out['z']          = out.get('z',          pd.Series(defaults.get('z', 0.60), index=out.index)).fillna(defaults.get('z', 0.60))
    out['y_CO2']      = out.get('y_CO2',      pd.Series(defaults.get('y_CO2', 0.01435), index=out.index)).fillna(defaults.get('y_CO2', 0.01435))
    out['y_H2O']      = out.get('y_H2O',      pd.Series(defaults.get('y_H2O', 0.00002), index=out.index)).fillna(defaults.get('y_H2O', 0.00002))
    out['U_ms']       = out.get('U_ms',       pd.Series(defaults.get('U_ms', 1.0), index=out.index)).fillna(defaults.get('U_ms', 1.0))
    out['diameter_m'] = out.get('diameter_m', pd.Series(defaults.get('diameter_m', 1.184), index=out.index)).fillna(defaults.get('diameter_m', 1.184))

    # Angle to sin/cos (avoid circular discontinuity)
    ang = np.deg2rad(out['angle_deg'].astype(float))
    out['angle_sin'] = np.sin(ang)
    out['angle_cos'] = np.cos(ang)

    # Dew point → water_flag
    water_flag, T_dp = compute_water_flag(out['pressure_bar'], out['y_H2O'], out['temperature_C'], margin_C=2.0)
    out['dew_point_C'] = T_dp
    out['water_flag']  = water_flag

    # Physics baseline features (fugacity, CR, CMT, C_phys)
    f_mpa = f_co2_mpa(out['pressure_bar'], out['y_CO2'], out['z'])
    T_K   = out['temperature_C'] + 273.15

    out['fco2_mpa'] = f_mpa
    out['CR_mm_y']  = C_R_star_mm_y(T_K, f_mpa)                   # with default gamma
    out['CMT_mm_y'] = C_MT_simple_mm_y(out['U_ms'], out['diameter_m'], f_mpa)

    if params_phys:
        out['C_phys_mm_y'] = C_phys_mm_y(
            T_K, f_mpa, out['water_flag'], out['U_ms'], out['diameter_m'],
            gamma=params_phys.get('gamma', 0.03),
            c1=params_phys.get('c1', 0.2),
            b=params_phys.get('b', 0.7),
            c=params_phys.get('c', 0.0),
            betaMT=params_phys.get('betaMT', 0.4)
        )
    else:
        out['C_phys_mm_y'] = C_phys_mm_y(
            T_K, f_mpa, out['water_flag'], out['U_ms'], out['diameter_m']
        )

    return out

def train_cgr_model(train_df, target_col='depth_cgr'):
    """
    Trains an ML model to predict depth CGR using engineered features + physics baseline.
    Uses sequential (latitude, longitude) split to avoid spatial leakage.
    """
    # Sort spatially (no random shuffle)
    df = train_df.sort_values(['latitude', 'longitude']).reset_index(drop=True)

    # Feature columns
    num_cols = [
        'latitude','longitude','angle_sin','angle_cos',
        'depth','length','width',
        'temperature_C','pressure_bar',
        'fco2_mpa','CR_mm_y','CMT_mm_y','C_phys_mm_y'
    ]
    cat_cols = ['feature_identification','anomaly_dim_class']

    # Preprocess
    pre = ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ])

    
    
    reg = XGBRegressor(
        n_estimators=500, learning_rate=0.06, max_depth=5,
        subsample=0.9, colsample_bytree=0.9, random_state=42
    )
    

    pipe = Pipeline([('pre', pre), ('reg', reg)])

    
    split_idx = int(len(df)*0.75)
    train, test = df.iloc[:split_idx], df.iloc[split_idx:]

    X_train = train[num_cols + cat_cols]
    y_train = train[target_col].astype(float)
    X_test  = test[num_cols + cat_cols]
    y_test  = test[target_col].astype(float)

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    print(f"CGR model — MAE={mae:.4f} mm/y, R²={r2:.3f}")
    return pipe


def train_and_predict_10y(df_train, df_predict,
                          params_phys=None,
                          defaults=None,
                          horizon_years=10.0):
    """
    df_train: rows with known depth CGR (mm/y). If you have matched 2011→2022, add length/width CGR too.
    df_predict: rows for which you want 10y future geometry (same columns, with current depth/len/width & conditions).
    params_phys: optional dict {gamma,c1,b,c,betaMT} for physics baseline; else defaults are used.
    defaults: optional dict for missing z, y_CO2, y_H2O, U_ms, diameter_m.
    """

    # 1) Engineer features (physics + angle encodings)
    train_fe = add_engineered_features(df_train, params_phys=params_phys, defaults=defaults)
    pred_fe  = add_engineered_features(df_predict, params_phys=params_phys, defaults=defaults)

    # 2) Train CGR model
    assert 'depth_cgr' in train_fe.columns, "df_train must include depth_cgr (target)."
    cgr_model = train_cgr_model(train_fe, target_col='depth_cgr')

    # 3) Predict CGR on df_predict
    feat_cols = [
        'latitude','longitude','angle_sin','angle_cos',
        'depth','length','width',
        'temperature_C','pressure_bar',
        'fco2_mpa','CR_mm_y','CMT_mm_y','C_phys_mm_y',
        'feature_identification','anomaly_dim_class'
    ]
    pred_cgr = cgr_model.predict(pred_fe[feat_cols])
    pred_cgr = np.clip(pred_cgr, 0.0, None)  # no negative growth for future



    # 6) Future geometry after 10 years
    out = pred_fe.copy()
    out['pred_cgr_depth_mm_y'] = pred_cgr
    # out['pred_cgr_length_mm_y'] = pred_len_cgr
    # out['pred_cgr_width_mm_y']  = pred_wid_cgr

    out['pred_depth_mm_10y']  = np.clip(out['depth']  + horizon_years * pred_cgr,     0.0, None)
    # out['pred_length_mm_10y'] = np.clip(out['length_mm'] + horizon_years * pred_len_cgr, 0.0, None)
    # out['pred_width_mm_10y']  = np.clip(out['width_mm']  + horizon_years * pred_wid_cgr, 0.0, None)

    # Keep ID columns aligned
    keep_cols = [
        'latitude','longitude','angle_deg',
        'feature_identification','anomaly_dim_class',
        'depth','length','width',
        'temperature_C','pressure_bar',
        'pred_cgr_depth_mm_y',#'pred_cgr_length_mm_y','pred_cgr_width_mm_y',
        'pred_depth_mm_10y',#'pred_length_mm_10y','pred_width_mm_10y'
    ]
    return out[keep_cols].reset_index(drop=True)


In [7]:
# df_train needs: your current geometry + conditions + labels:
#   depth_cgr_mm_per_y   (for new anomalies: depth_2022/years_between)
# Optionally (if you have matched 2011→2022 pairs):
#   length_cgr_mm_per_y, width_cgr_mm_per_y

# df_predict needs: same inputs but no CGR labels (we predict them)

params_phys = dict(gamma=0.03, c1=0.2, b=0.7, c=0.0, betaMT=0.4)  # or leave None
defaults    = dict(z=0.60, y_CO2=0.01435, y_H2O=0.002, U_ms=1.0, diameter_m=1.184)

df_future = train_and_predict_10y(df_train =df , df_predict = df,
                                  params_phys=params_phys,
                                  defaults=defaults,
                                  horizon_years=1.0)

df_future.head()


CGR model — MAE=0.1461 mm/y, R²=-0.086


,latitude,longitude,angle_deg,feature_identification,anomaly_dim_class,depth,length,width,temperature_C,pressure_bar,pred_cgr_depth_mm_y,pred_depth_mm_10y
0,16.736072,82.293731,120.5,Corrosion,PINH,2.54,15.0,18.0,30.8,92,0.042732,2.582732
1,16.736026,82.293298,254.0,Pipe mill anomaly,AXSL,2.54,40.0,23.0,30.8,92,0.041141,2.581141
2,16.735664,82.289963,169.0,Pipe mill anomaly,AXSL,2.54,60.0,18.0,30.8,92,0.632936,3.172936
3,16.736874,82.286373,71.5,Pipe mill anomaly,PINH,2.54,22.0,23.0,30.8,92,0.007490,2.547490
4,16.736874,82.286373,71.5,Pipe mill anomaly,PINH,2.54,22.0,23.0,30.8,92,0.007490,2.547490


### Considering pressure, temperature and concentration of change of Co2 and H2O with time

In [40]:
import pandas as pd
import numpy as np

years = 10
months = years * 12

# Seasonal oscillation as example
t = np.arange(months)
pressure_series = 90 + 5 * np.sin(2 * np.pi * t / 12)  # oscillates between 85–95 bar
temp_series = 30 + 3 * np.sin(2 * np.pi * t / 12)      # oscillates between 27–33 °C
yCO2_series = 0.012 + 0.003 * np.sin(2 * np.pi * t / 12)
yH2O_series = 0.00002 + 0.0005 * np.cos(2 * np.pi * t / 6)

scenario = pd.DataFrame({
    'pressure_bar': pressure_series,
    'temperature_C': temp_series,
    'y_CO2': yCO2_series,
    'y_H2O': yH2O_series
})


In [41]:
def simulate_variable_growth(
    df_points,
    model_cgr,
    years=10,
    monthly=True,
    defaults=None,
    params_phys=None,
    scenario=None
):
    """
    Simulates time-varying corrosion depth with changing
    pressure, temperature, CO2 and H2O mole fractions.

    scenario: DataFrame with columns
        ['pressure_bar','temperature_C','y_CO2','y_H2O']
        length = n_steps (months or years)
    """

    steps_per_year = 12 if monthly else 1
    total_steps = years * steps_per_year
    dt = 1.0 / steps_per_year

    depth = df_points['depth'].copy().to_numpy()
    depth_trajectory = [depth.copy()]

    for t in range(total_steps):
        # Get scenario values (cycle or repeat if shorter)
        row = scenario.iloc[t % len(scenario)]
        p  = row['pressure_bar']
        T  = row['temperature_C']
        yCO2 = row['y_CO2']
        yH2O = row['y_H2O']

        # Update input features
        df_step = df_points.copy()
        df_step['pressure_bar'] = p
        df_step['temperature_C'] = T
        df_step['y_CO2'] = yCO2
        df_step['y_H2O'] = yH2O
        df_step['depth_mm'] = depth

        # Recompute engineered features
        df_step = add_engineered_features(df_step, params_phys=params_phys, defaults=defaults)

        # Predict CGR
        feat_cols = [
            'latitude','longitude','angle_sin','angle_cos',
            'depth','length','width',
            'temperature_C','pressure_bar',
            'fco2_mpa','CR_mm_y','CMT_mm_y','C_phys_mm_y',
            'feature_identification','anomaly_dim_class'
        ]
        cgr_step = model_cgr.predict(df_step[feat_cols])
        cgr_step = np.clip(cgr_step, 0.0, None)

        # Update depth
        depth += cgr_step * dt
        depth_trajectory.append(depth.copy())

    # Construct output time series
    depth_matrix = np.vstack(depth_trajectory)
    
    time_axis = np.arange(len(depth_trajectory)) * dt
    
    result = pd.DataFrame(depth_matrix,columns=[f"point_{i}" for i in range(len(df_points))]
    )
    result['time_years'] = time_axis
    return result


In [42]:
params_phys = dict(gamma=0.03, c1=0.2, b=0.7, c=0.0, betaMT=0.4)  # or leave None
defaults    = dict(z=0.60, y_CO2=0.01435, y_H2O=0.00002, U_ms=1.0, diameter_m=1.184)

df_future = train_and_predict_10y(df_train =df, df_predict = df,
                                  params_phys=params_phys,
                                  defaults=defaults,
                                  horizon_years=1.0)

df_future.head()

CGR model — MAE=0.1461 mm/y, R²=-0.086


,latitude,longitude,angle_deg,feature_identification,anomaly_dim_class,depth,length,width,temperature_C,pressure_bar,pred_cgr_depth_mm_y,pred_depth_mm_10y
0,16.736072,82.293731,120.5,Corrosion,PINH,2.54,15.0,18.0,30.8,92,0.042732,2.582732
1,16.736026,82.293298,254.0,Pipe mill anomaly,AXSL,2.54,40.0,23.0,30.8,92,0.041141,2.581141
2,16.735664,82.289963,169.0,Pipe mill anomaly,AXSL,2.54,60.0,18.0,30.8,92,0.632936,3.172936
3,16.736874,82.286373,71.5,Pipe mill anomaly,PINH,2.54,22.0,23.0,30.8,92,0.007490,2.547490
4,16.736874,82.286373,71.5,Pipe mill anomaly,PINH,2.54,22.0,23.0,30.8,92,0.007490,2.547490


In [43]:
df_train_fe = add_engineered_features(
    df,
    params_phys=params_phys,
    defaults=defaults
)

model_cgr = train_cgr_model(
    df_train_fe,
    target_col='depth_cgr'
)

CGR model — MAE=0.1461 mm/y, R²=-0.086


In [44]:
result = simulate_variable_growth(df, model_cgr = model_cgr, years=5, monthly=False,
                                  scenario=scenario, params_phys=params_phys)

result # First few time steps
# print(result.tail())   # Last few time steps


,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,point_1097,point_1098,point_1099,point_1100,point_1101,point_1102,point_1103,point_1104,point_1105,time_years
0,2.540000,2.540000,2.540000,2.54000,2.54000,1.720000,1.720000,1.720000,2.070000,2.070000,...,2.070000,1.720000,1.720000,1.720000,1.720000,1.720000,1.720000,1.720000,1.720000,0.0
1,2.582732,2.581141,3.172936,2.54749,2.54749,1.729523,1.734360,1.742960,2.158514,2.132067,...,2.596122,2.153158,2.152809,2.149055,2.148653,2.129275,2.136398,2.137227,2.155690,1.0
2,2.625465,2.622283,3.805873,2.55498,2.55498,1.739045,1.748720,1.765921,2.247028,2.194135,...,3.122244,2.586316,2.585618,2.578110,2.577306,2.538551,2.552795,2.554454,2.591381,2.0
3,2.668197,2.663424,4.438809,2.56247,2.56247,1.748568,1.763080,1.788881,2.335542,2.256202,...,3.648366,3.019473,3.018426,3.007165,3.005959,2.947826,2.969193,2.971680,3.027071,3.0
4,2.710929,2.704565,5.071746,2.56996,2.56996,1.758090,1.777440,1.811842,2.424055,2.318269,...,4.174489,3.452631,3.451235,3.436220,3.434613,3.357102,3.385590,3.388907,3.462761,4.0
5,2.753662,2.745707,5.704682,2.57745,2.57745,1.767613,1.791799,1.834802,2.512569,2.380337,...,4.700611,3.885789,3.884044,3.865275,3.863266,3.766377,3.801988,3.806134,3.898452,5.0


### UPdated hybrid model : January

In [8]:
import numpy as np
import pandas as pd


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GroupKFold
from xgboost import XGBRegressor




def dew_point_C_from_pH2O_kPa(p_h2o_kpa):
    p = np.clip(np.asarray(p_h2o_kpa, dtype=float), 1e-6, None)
    ln_arg = np.log(p / 0.61078)
    return (237.3 * ln_arg) / (17.27 - ln_arg)

def compute_water_flag(P_bar, y_H2O, T_wall_C, margin_C=2.0):
    p_h2o_kpa = np.clip(y_H2O, 0.0, 1.0) * np.clip(P_bar, 0.0, None) * 100.0  # bar→kPa
    T_dp = dew_point_C_from_pH2O_kPa(p_h2o_kpa)
    return (np.asarray(T_wall_C) <= (T_dp - margin_C)).astype(int), T_dp

def f_co2_mpa(P_bar, y_CO2, z):
    phi = np.clip(np.asarray(z), 0.3, 1.2)
    P_mpa = np.clip(np.asarray(P_bar), 0.0, None) / 10.0  # bar→MPa
    return np.clip(phi * np.asarray(y_CO2) * P_mpa, 1e-5, None)

def T_scale_K(f_mpa):
    f = np.clip(np.asarray(f_mpa), 1e-5, None)
    return 2400.0 / (6.7 + 0.61*np.log10(f))

def C_R_star_mm_y(T_K, f_mpa, gamma=0.03, A_log=1710.0, C0_log=5.8, fexp_log=0.67):
    T = np.clip(np.asarray(T_K), 1e-6, None)
    f = np.clip(np.asarray(f_mpa), 1e-5, None)
    base_log10 = C0_log - A_log/T + fexp_log*np.log10(f)
    CR = 10.0**base_log10
    decay = np.exp(-gamma * np.maximum(T - T_scale_K(f), 0.0))
    return CR * decay

def C_MT_simple_mm_y(U_ms, diameter_m, f_mpa, c1=0.2, b=0.7, c=0.0, betaMT=0.4):
    U = np.clip(np.asarray(U_ms), 1e-6, None)
    d = np.clip(np.asarray(diameter_m), 1e-6, None)
    f = np.clip(np.asarray(f_mpa), 1e-5, None)
    return c1 * (U**b) * (d**c) * (f**betaMT)

def C_phys_mm_y(T_K, f_mpa, water_flag, U_ms, diameter_m,
                gamma=0.03, c1=0.2, b=0.7, c=0.0, betaMT=0.4,
                CpH=1.0, Cscale=1.0, Ctau=1.0):
    CR  = C_R_star_mm_y(T_K, f_mpa, gamma=gamma)
    CMT = C_MT_simple_mm_y(U_ms, diameter_m, f_mpa, c1=c1, b=b, c=c, betaMT=betaMT)
    CR  = np.clip(CR,  1e-12, None)
    CMT = np.clip(CMT, 1e-12, None)
    Ccomb = 1.0 / (1.0/CR + 1.0/CMT)
    return Ccomb * np.asarray(water_flag) * CpH * Cscale * Ctau


# 1 )Feature engineering

def add_engineered_features(df, params_phys=None, defaults=None):
    """
    Adds physics & encodings needed by the ML model.
    params_phys: dict with gamma, c1, b, c, betaMT (optional, to compute C_phys baseline as a feature)
    defaults: dict with fallback values for z, y_CO2, y_H2O, U_ms, diameter_m
    """
    out = df.copy()

    # Defaults (you can pass pipeline-level constants here)
    defaults = defaults or {}
    out['z']          = out.get('z',          pd.Series(defaults.get('z', 0.60), index=out.index)).fillna(defaults.get('z', 0.60))
    out['y_CO2']      = out.get('y_CO2',      pd.Series(defaults.get('y_CO2', 0.01435), index=out.index)).fillna(defaults.get('y_CO2', 0.01435))
    out['y_H2O']      = out.get('y_H2O',      pd.Series(defaults.get('y_H2O', 0.00002), index=out.index)).fillna(defaults.get('y_H2O', 0.00002))
    out['U_ms']       = out.get('U_ms',       pd.Series(defaults.get('U_ms', 1.0), index=out.index)).fillna(defaults.get('U_ms', 1.0))
    out['diameter_m'] = out.get('diameter_m', pd.Series(defaults.get('diameter_m', 1.184), index=out.index)).fillna(defaults.get('diameter_m', 1.184))

    # Angle to sin/cos (avoid circular discontinuity)
    ang = np.deg2rad(out['angle_deg'].astype(float))
    out['angle_sin'] = np.sin(ang)
    out['angle_cos'] = np.cos(ang)

    # Dew point → water_flag
    water_flag, T_dp = compute_water_flag(out['pressure_bar'], out['y_H2O'], out['temperature_C'], margin_C=2.0)
    out['dew_point_C'] = T_dp
    out['water_flag']  = water_flag

    # Physics baseline features (fugacity, CR, CMT, C_phys)
    f_mpa = f_co2_mpa(out['pressure_bar'], out['y_CO2'], out['z'])
    T_K   = out['temperature_C'] + 273.15

    out['fco2_mpa'] = f_mpa
    out['CR_mm_y']  = C_R_star_mm_y(T_K, f_mpa)                   # with default gamma
    out['CMT_mm_y'] = C_MT_simple_mm_y(out['U_ms'], out['diameter_m'], f_mpa)

    if params_phys:
        out['C_phys_mm_y'] = C_phys_mm_y(
            T_K, f_mpa, out['water_flag'], out['U_ms'], out['diameter_m'],
            gamma=params_phys.get('gamma', 0.03),
            c1=params_phys.get('c1', 0.2),
            b=params_phys.get('b', 0.7),
            c=params_phys.get('c', 0.0),
            betaMT=params_phys.get('betaMT', 0.4)
        )
    else:
        out['C_phys_mm_y'] = C_phys_mm_y(
            T_K, f_mpa, out['water_flag'], out['U_ms'], out['diameter_m']
        )

    return out

# ==========================================================
# 2) Train CGR model (spatial split)
# ==========================================================
def train_cgr_model(train_df, target_col='depth_cgr'):
    """
    Trains an ML model to predict depth CGR using engineered features + physics baseline.
    Uses sequential (latitude, longitude) split to avoid spatial leakage.
    """
    # Sort spatially (no random shuffle)
    df = train_df.sort_values(['latitude', 'longitude']).reset_index(drop=True)

    # Feature columns
    num_cols = [
        'latitude','longitude','angle_sin','angle_cos',
        'depth','length','width',
        'temperature_C','pressure_bar',
        'fco2_mpa','CR_mm_y','CMT_mm_y','C_phys_mm_y'
    ]
    cat_cols = ['feature_identification','anomaly_dim_class']

    # Preprocess
    pre = ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ])

    
    
    reg = XGBRegressor(
        n_estimators=500, learning_rate=0.06, max_depth=5,
        subsample=0.9, colsample_bytree=0.9, random_state=42
    )
    

    pipe = Pipeline([('pre', pre), ('reg', reg)])

    
    split_idx = int(len(df)*0.75)
    train, test = df.iloc[:split_idx], df.iloc[split_idx:]

    X_train = train[num_cols + cat_cols]
    y_train = train[target_col].astype(float)
    X_test  = test[num_cols + cat_cols]
    y_test  = test[target_col].astype(float)

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    print(f"CGR model — MAE={mae:.4f} mm/y, R²={r2:.3f}")
    return pipe, {
    "y_train": y_train,
    "y_test": y_test,
    "y_pred": y_pred,
    "test_df": test
}



def train_and_predict_10y(df_train, df_predict,
                          params_phys=None,
                          defaults=None,
                          horizon_years=10.0):
    """
    df_train: rows with known depth CGR (mm/y). If you have matched 2011→2022, add length/width CGR too.
    df_predict: rows for which you want 10y future geometry (same columns, with current depth/len/width & conditions).
    params_phys: optional dict {gamma,c1,b,c,betaMT} for physics baseline; else defaults are used.
    defaults: optional dict for missing z, y_CO2, y_H2O, U_ms, diameter_m.
    """

    # 1) Engineer features (physics + angle encodings)
    train_fe = add_engineered_features(df_train, params_phys=params_phys, defaults=defaults)
    pred_fe  = add_engineered_features(df_predict, params_phys=params_phys, defaults=defaults)

    # 2) Train CGR model
    assert 'depth_cgr' in train_fe.columns, "df_train must include depth_cgr (target)."
    cgr_model = train_cgr_model(train_fe, target_col='depth_cgr')

    # 3) Predict CGR on df_predict
    feat_cols = [
        'latitude','longitude','angle_sin','angle_cos',
        'depth','length','width',
        'temperature_C','pressure_bar',
        'fco2_mpa','CR_mm_y','CMT_mm_y','C_phys_mm_y',
        'feature_identification','anomaly_dim_class'
    ]
    # ML-predicted CGR
    pred_cgr_ml = cgr_model.predict(pred_fe[feat_cols])
    pred_cgr_ml = np.clip(pred_cgr_ml, 0.0, None)
    
    # Physics CGR (already computed in features)
    pred_cgr_phys = pred_fe['C_phys_mm_y'].values
    
    # Physics-anchored blending (NEW)
    pred_cgr = (
        (1.0 - lambda_phys) * pred_cgr_ml
        + lambda_phys * pred_cgr_phys
    )
    
    pred_cgr = np.clip(pred_cgr, 0.0, None)




    # 6) Future geometry after 10 years
    out = pred_fe.copy()
    out['pred_cgr_depth_ml_mm_y']   = pred_cgr_ml
    out['pred_cgr_depth_phys_mm_y'] = pred_cgr_phys
    out['pred_cgr_depth_mm_y']      = pred_cgr


    out['pred_depth_mm_10y']  = np.clip(out['depth']  + horizon_years * pred_cgr,     0.0, None)
   
    keep_cols = [
        'latitude','longitude','angle_deg',
        'feature_identification','anomaly_dim_class',
        'depth','length','width',
        'temperature_C','pressure_bar',
        'pred_cgr_depth_mm_y',
        'pred_depth_mm_10y',
    ]
    return out[keep_cols].reset_index(drop=True)


In [9]:
import pandas as pd
import numpy as np

years = 10
months = years * 12

# Seasonal oscillation as example
t = np.arange(months)
pressure_series = 90 + 5 * np.sin(2 * np.pi * t / 12)  # oscillates between 85–95 bar
temp_series = 30 + 3 * np.sin(2 * np.pi * t / 12)      # oscillates between 27–33 °C
yCO2_series = 0.012 + 0.003 * np.sin(2 * np.pi * t / 12)
yH2O_series = 0.00002 + 0.0005 * np.cos(2 * np.pi * t / 6)

scenario = pd.DataFrame({
    'pressure_bar': pressure_series,
    'temperature_C': temp_series,
    'y_CO2': yCO2_series,
    'y_H2O': yH2O_series
})


In [10]:
def simulate_variable_growth(
    df_points,
    model_cgr,
    years=10,
    monthly=True,
    defaults=None,
    params_phys=None,
    scenario=None,
    lambda_phys=0.2
):
    """
    Simulates time-varying corrosion depth with changing
    pressure, temperature, CO2 and H2O mole fractions.

    scenario: DataFrame with columns
        ['pressure_bar','temperature_C','y_CO2','y_H2O']
        length = n_steps (months or years)
    """

    steps_per_year = 12 if monthly else 1
    total_steps = years * steps_per_year
    dt = 1.0 / steps_per_year

    depth = df_points['depth'].copy().to_numpy()
    depth_trajectory = [depth.copy()]

    for t in range(total_steps):
        # Get scenario values (cycle or repeat if shorter)
        row = scenario.iloc[t % len(scenario)]
        p  = row['pressure_bar']
        T  = row['temperature_C']
        yCO2 = row['y_CO2']
        yH2O = row['y_H2O']

        # Update input features
        df_step = df_points.copy()
        df_step['pressure_bar'] = p
        df_step['temperature_C'] = T
        df_step['y_CO2'] = yCO2
        df_step['y_H2O'] = yH2O
        df_step['depth'] = depth

        # Recompute engineered features
        df_step = add_engineered_features(df_step, params_phys=params_phys, defaults=defaults)

        # Predict CGR
        feat_cols = [
            'latitude','longitude','angle_sin','angle_cos',
            'depth','length','width',
            'temperature_C','pressure_bar',
            'fco2_mpa','CR_mm_y','CMT_mm_y','C_phys_mm_y',
            'feature_identification','anomaly_dim_class'
        ]
        # ML prediction
        cgr_ml = model_cgr.predict(df_step[feat_cols])
        cgr_ml = np.clip(cgr_ml, 0.0, None)
        
        # Physics CGR (already recomputed)
        cgr_phys = df_step['C_phys_mm_y'].to_numpy()
        
        # Blended CGR (NEW CONCEPT)
        cgr_step = (
            (1.0 - lambda_phys) * cgr_ml
            + lambda_phys * cgr_phys
        )
        
        cgr_step = np.clip(cgr_step, 0.0, None)

        # Update depth
        depth += cgr_step * dt
        depth_trajectory.append(depth.copy())

    # Construct output time series
    depth_matrix = np.vstack(depth_trajectory)
    
    time_axis = np.arange(len(depth_trajectory)) * dt
    
    result = pd.DataFrame(depth_matrix,columns=[f"point_{i}" for i in range(len(df_points))]
    )
    result['time_years'] = time_axis
    return result


In [13]:

# 1) Feature engineering for training (uses CGR_obs)

df_train_fe = add_engineered_features(
    df,
    params_phys=params_phys,
    defaults=defaults
)


# 2) Train CGR ML model (target = observed CGR)

model_cgr, diag = train_cgr_model(
    df_train_fe,
    target_col='depth_cgr'
)


# 3) Physics-anchored time simulation (NO CGR_obs used here)

lambda_phys = 0.2  # physics anchoring strength (tune if needed)

result = simulate_variable_growth(
    df_points=df,
    model_cgr=model_cgr,
    years=5,
    monthly=False,
    scenario=scenario,
    params_phys=params_phys,
    defaults=defaults,
    lambda_phys=lambda_phys
)


result.head()      # first few time steps



CGR model — MAE=0.1461 mm/y, R²=-0.086


,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,point_1097,point_1098,point_1099,point_1100,point_1101,point_1102,point_1103,point_1104,point_1105,time_years
0,2.540000,2.540000,2.540000,2.540000,2.540000,1.720000,1.720000,1.720000,2.070000,2.070000,...,2.070000,1.720000,1.720000,1.720000,1.720000,1.720000,1.720000,1.720000,1.720000,0.0
1,2.574186,2.572913,3.046349,2.545992,2.545992,1.727618,1.731488,1.738368,2.140811,2.119654,...,2.490898,2.066526,2.066247,2.063244,2.062923,2.047420,2.053118,2.053781,2.068552,1.0
2,2.608372,2.605826,3.571092,2.551984,2.551984,1.735236,1.742976,1.756737,2.211622,2.169308,...,2.984136,2.416054,2.424168,2.412408,2.397422,2.365661,2.369218,2.335947,2.382729,2.0
3,2.642558,2.638739,4.095835,2.557976,2.557976,1.742854,1.754464,1.775105,2.282433,2.218962,...,3.490585,2.760358,2.781301,2.771591,2.743990,2.687708,2.696654,2.631013,2.695674,3.0
4,2.676743,2.671652,4.620579,2.563968,2.563968,1.750472,1.765952,1.793473,2.355508,2.268616,...,3.997034,3.114859,3.152244,3.141439,3.100681,3.003510,3.036215,2.916162,3.009833,4.0


# Testing various error issues

In [16]:
y_test  = diag["y_test"]
y_train = diag["y_train"]
y_pred  = diag["y_pred"]
test_df = diag["test_df"]


y_mean = np.full_like(y_test, y_train.mean())
y_phys = test_df['C_phys_mm_y'].values

print("Mean baseline MAE:", mean_absolute_error(y_test, y_mean))
print("Physics MAE:", mean_absolute_error(y_test, y_phys))
print("ML MAE:", mean_absolute_error(y_test, y_pred))


Mean baseline MAE: 0.11838242652406229
Physics MAE: 0.31922305760354713
ML MAE: 0.14607349901558958


In [14]:
df['depth_cgr'].describe()

count    1106.000000
mean        0.393408
std         0.198833
min         0.000000
25%         0.430000
50%         0.430000
75%         0.517500
max         0.762000
Name: depth_cgr, dtype: float64

### Dashboard